# ch4_12 python connect
이제 python에서 RDBMS를 사용하는 방법을 알아보겠습니다. 먼저 python에서 RDMBS를 사용하기 위한 패키지를 설치해야합니다. 종류는 여러가지가 있지만, 가장 대표적인 pymysql을 사용해보겠습니다. 이런 종류의 라이브러리들을 보통 DB driver라고 부릅니다.

## pymysql을 이용해서 RDBMS 접속하기

In [1]:
!pip install pymysql

이제 pymysql로 DB에 접속해보겠습니다. 접속을 관리하는 conn (connection의 약자입니다. 보통 이렇게 줄여서 많이 사용합니다.) 객체를 만들어서 사용합니다.

In [47]:
import pymysql 

conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="olist"
)

In [48]:
conn, type(conn)

(<pymysql.connections.Connection at 0x1480f49a0>,
 pymysql.connections.Connection)

pymysql 라이브러리 안에 정의되어 있는 Connection 클래스로 만든 객체라는 소리입니다. 

## Read 쿼리 날리기
이제 이 conn으로 쿼리를 날려보겠습니다. 쿼리를 날리기 위해서는 Connection 클래스 안에 정의되어 있는 cursor를 사용해야 합니다.

쿼리를 날리기 위해서는 먼저 SQL 문을 문자열로 만들어야 합니다. 그 다음, 마치 파일을 열고 데이터를 읽어오듯 cursor 객체를 만든 뒤에 execute 함수에 sql문을 파라미터로 전달해서 쿼리를 날리면 됩니다. 그 다음, fetchall을 이용해서 결과를 읽어오면 됩니다.

In [49]:
def read_all_items():
    select_sql = "SELECT * FROM items WHERE id=1"
    with conn.cursor() as cursor:
        cursor.execute(select_sql)
        result = cursor.fetchall()
    return result

In [50]:
records = read_all_items()
print(records)

ProgrammingError: (1146, "Table 'olist.items' doesn't exist")

### 연습 문제
- titanic 테이블에서 Pclass, Survived, Name, Age 컬럼을 Age 오름차순으로 10개만 읽어와서 print 문으로 출력해보세요.

## Write 쿼리 날리기
write 쿼리는 read 쿼리와 다르게 execute로 쿼리를 실행한 다음, commit을 해주어야 결과가 반영이 됩니다. 이를 온전히 이해하기 위해서는 transaction 개념을 이해해야 하는데, 일단은 넘어가고 데이터를 보호하기 위한 안전장치라고 생각하고 넘어갑시다.

products 테이블에 새로운 레코드를 하나 추가해보겠습니다. 이 때, python for문을 사용해서 편하게 SQL문을 작성해보겠습니다.

In [16]:
new_products = [
    ("smash", 68000, "puma"),
    ("spark", 12200, "puma"),
    ("allstar", 98000, "converse"),
]

In [17]:
def generate_insert_sql(new_products):
    insert_sql = "INSERT INTO items(name, price, brand) VALUES "
    for name, price, description in new_products:
        insert_sql += f"('{name}', {price}, '{description}'), "
    # 마지막에 붙은 , 떼주기
    insert_sql = insert_sql[:-2]
    return insert_sql

In [18]:
insert_sql = generate_insert_sql(new_products)
insert_sql

"INSERT INTO items(name, price, brand) VALUES ('smash', 68000, 'puma'), ('spark', 12200, 'puma'), ('allstar', 98000, 'converse')"

In [23]:
with conn.cursor() as cursor:
    cursor.execute(insert_sql)
    conn.commit()

In [24]:
read_all_items()

((1, 'superstar', 23400, 'adidas'),
 (2, 'easyboost', 58250, 'adidas'),
 (3, 'airforce', 89000, 'nike'),
 (4, 'jordan', 45000, 'nike'),
 (5, 'airbreaker', 89000, 'newbalance'),
 (6, 'booster', 69800, 'newbalance'),
 (7, 'smash', 68000, 'puma'),
 (8, 'spark', 12200, 'puma'),
 (9, 'allstar', 98000, 'converse'),
 (10, 'smash', 68000, 'puma'),
 (11, 'spark', 12200, 'puma'),
 (12, 'allstar', 98000, 'converse'))

### 대량의 데이터 일괄 쓰기

python 코드를 이용하여 CSV 파일을 읽어서 insert sql문을 작성한 다음, DB에 써줄 수 있었습니다. 그리고 여러 행에 걸쳐있는 데이터를 읽어와서 일괄적으로 insert를 수행할 수도 있습니다. 이를 이용해서 대량의 데이터를 CSV 라이브러리로 읽어와서 일정한 개수 단위로 끊어서 DB에 write를 할 수 있습니다. 

In [43]:
import csv
from tqdm import tqdm

def insert_multiple_values(insert_sql, values):
    with conn.cursor() as cursor:
        cursor.executemany(insert_sql, values)
        conn.commit()

def bulk_insert(csv_file_path, insert_sql, batch_size=1000):
    with open(csv_file_path) as fr:
        reader = csv.reader(fr)
        next(reader)
        buffer = []
        for row in tqdm(reader):
            buffer.append(tuple(row))
            if len(buffer) == batch_size:
                insert_multiple_values(
                    insert_sql, buffer)
                values = []
        if values:
            insert_multiple_values(insert_sql, buffer)

In [44]:
insert_sql = "INSERT INTO customers(customer_id, customer_unique_id, customer_zip_code_prefix, customer_city, customer_state) VALUES (%s, %s, %s, %s, %s)"
bulk_insert("./data/olist/customers.csv", insert_sql)

99441it [00:00, 917311.15it/s]


In [51]:
!pwd

/Users/user/dev/data_analytics_camp_2023_share/week4_database
